In [1]:
import requests

In [2]:
url = "https://cluster.elasticsearch.dataesr.ovh/bso-publications/_search"
headers = { "Authorization": open("../../.env_es_bsso", "r").read().strip() }
AGG_SIZE = 15
#LATEST_OBSERVATION_DATE = "2021Q2"

In [9]:
json = {
  "size": 0,
  "aggs" : {
    "observation_dates" : {
      "terms" : { "field": "observation_dates.keyword" }
    }
  }
}
obs_dates = requests.post(url, json=json, headers=headers).json()["aggregations"]["observation_dates"]["buckets"]
LATEST_OBSERVATION_DATE = obs_dates[0]['key']
print("LATEST_OBSERVATION_DATE", LATEST_OBSERVATION_DATE)

LATEST_OBSERVATION_DATE 2021Q2


### Sommaire

* [1. Publications](#publications)
    * [1.1. Général](#publi_general)
        * [1.1.1. Quelle est la dynamique d’ouverture de la santé en France ?](#dynamique_ouverture)
        * [1.1.2. Quelles sont les voies d’ouverture choisies pour les publications en santé ?](#voie_ouverture)
        * [1.1.3. Quelles sont les genres les plus ouverts en santé ?](#genre)
        * [1.1.4. Quelles sont les langues des productions ouvertes en santé ?](#langue)
        * [1.1.5. Quel impact le financement a-t-il sur l’ouverture des publications en santé ?](#financement)
        * [1.1.6. Quelle transparence dans la déclaration des conflits d'intérêts en santé ?](#coi)
    * [1.2. Les disciplines](#publi_disciplines)
        * [1.2.1. Quelle est la dynamique d’ouverture de la santé en fonction de ses disciplines ?](#dynamique_ouverture_disciplines)
        * [1.2.2. Quelles sont les voies d’ouverture choisies par les publications en fonction des disciplines ?](#voie_ouverture_disciplines)
    * [1.3. Les éditeurs/plateformes](#publi_editeurs)
        * [1.3.1. Quelle est la dynamique d’ouverture de la santé chez les éditeurs/plateformes ?](#dynamique_ouverture_editeurs)
        * [1.3.2. Quel type d’ouverture est majoritaire chez les éditeurs/plateformes en santé ?](#voie_ouverture_editeurs)
        * [1.3.3. Quelles sont les politiques d’ouverture des éditeurs/plateformes en santé ?](#politique_ouverture_editeurs)
        * [1.3.4. Quel est le poids des revues prédatrices dans la dynamique de science ouverte en santé ?](#predateur)
        * [1.3.5. Quelle est la répartition des licences utilisées chez les éditeurs/plateformes en santé ?](#licence)
        * [1.3.6. Quels sont les coûts des publications chez les éditeurs/plateformes en santé ?](#apc)
    * [1.4. Les archives](#publications_archives)
        * [1.4.1. Quelle est la dynamique d’ouverture de la santé parmi les archives ?](#publications_archives_ouverture)
        * [1.4.2. Quelles archives ouvertes sont les plus utilisées en santé ?](#publications_archives_utilisation)
        * [1.4.3. Quelle est la dynamique de dépôt par archive ouverte en santé ?](#publications_archives_dynamique)
        * [1.4.4. Quelle place occupe HAL dans la dynamique des archives ouvertes en santé ?](#publications_archives_hal)
    * [1.5. Les affiliations](#publications_affiliations)
        * [1.5.1. Taux d’ouverture des publications françaises, dans le domaine de la santé, par millésime tous types d’établissements confondus](#publications_affiliations_ouverture)
        * [1.5.2. Evolution du taux d’ouverture des publications en santé par types d’établissement entre millésimes](#publications_affiliations_evolution)
        * [1.5.3. Quel impact le pays d’affiliation des auteurs a-t-il sur le taux d’ouverture en santé ?](#publications_affiliations_impact)
        * [1.5.4. Classement des 10 pays d'affiliation des auteurs de rang utile avec lesquels la France collabore le plus, selon le taux d’accès ouvert de leurs publications en santé](#publications_affiliations_classement)

# 1. Publications <a class="anchor" id="publications"></a>

## 1.1. Général <a class="anchor" id="publi_general"></a>

### 1.1.1. Quelle est la dynamique d’ouverture de la santé en France ? (barchart + linechart)<a class="anchor" id="dynamique_ouverture"></a>

In [10]:
# Autant de requêtes que de observation date sont à faire
# exemple pour la dernière observation date
json = {
  "size": 0,
  "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "year"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
              }
            }
          }
    }
  }
}

results = requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']
results

[{'key': 2018,
  'doc_count': 167082,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 86166},
    {'key': 0, 'key_as_string': 'false', 'doc_count': 80916}]}},
 {'key': 2019,
  'doc_count': 165572,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 94454},
    {'key': 0, 'key_as_string': 'false', 'doc_count': 71118}]}},
 {'key': 2020,
  'doc_count': 163739,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 94184},
    {'key': 0, 'key_as_string': 'false', 'doc_count': 69555}]}},
 {'key': 2016,
  'doc_count': 155772,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 81394},
    {'key': 1, 'key_as_string': 'true', 'doc_count':

pour le 1er graphique (barre), pour chaque date d'observation, on calcule le ratio d'ouverture pour la publication_date de l'année précédente.
Donc, pour observation_date = 2021Q1, on calcule le ratio d'ouvertue de la publication_date 2020 : nb de oa true / (nb true + nb false)

In [11]:
tmp = [result for result in results if result['key'] == 2020][0]['by_is_oa']['buckets']
score_true = [result for result in tmp if result['key'] == 1][0]['doc_count']
score_false = [result for result in tmp if result['key'] == 0][0]['doc_count']
total = score_true + score_false
response = score_true / total * 100
response

57.52081055826651

### 1.1.2. Quelles sont les voies d’ouverture choisies pour les publications en santé ? (stacked barchart + stacked area graph + treemap) <a class="anchor" id="voie_ouverture"></a>

In [12]:
json = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type.keyword",
            "missing": "N/A"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167082,
  'by_oa_host_type': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 80916},
    {'key': 'publisher;repository', 'doc_count': 33752},
    {'key': 'repository', 'doc_count': 28804},
    {'key': 'publisher', 'doc_count': 23610}]}},
 {'key': 2019,
  'doc_count': 165572,
  'by_oa_host_type': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 71118},
    {'key': 'publisher;repository', 'doc_count': 37679},
    {'key': 'publisher', 'doc_count': 28514},
    {'key': 'repository', 'doc_count': 28261}]}},
 {'key': 2020,
  'doc_count': 163739,
  'by_oa_host_type': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 69555},
    {'key': 'publisher;repository', 'doc_count': 40677},
    {'key': 'repository', 'doc_count': 28517},
    {'key': 'publisher', 'doc_count': 24990}]}},
 {'key': 2

### 1.1.3. Quelles sont les genres les plus ouverts en santé ? <a class="anchor" id="genre"></a>

In [13]:
json = {
  "size": 0,
  "aggs": {
    "by_is_oa": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
      },
      "aggs": {
        "by_publication_genre": {
          "terms": {
            "field": "genre.keyword"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_is_oa']['buckets']

[{'key': 0,
  'key_as_string': 'false',
  'doc_count': 654731,
  'by_publication_genre': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 1040,
   'buckets': [{'key': 'journal-article', 'doc_count': 490859},
    {'key': 'book-chapter', 'doc_count': 83475},
    {'key': 'proceedings-article', 'doc_count': 53751},
    {'key': 'dataset', 'doc_count': 6540},
    {'key': 'other', 'doc_count': 5793},
    {'key': 'book', 'doc_count': 4438},
    {'key': 'component', 'doc_count': 3125},
    {'key': 'monograph', 'doc_count': 2275},
    {'key': 'posted-content', 'doc_count': 2165},
    {'key': 'reference-entry', 'doc_count': 1126}]}},
 {'key': 1,
  'key_as_string': 'true',
  'doc_count': 642317,
  'by_publication_genre': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 1175,
   'buckets': [{'key': 'journal-article', 'doc_count': 561426},
    {'key': 'proceedings-article', 'doc_count': 29801},
    {'key': 'book-chapter', 'doc_count': 27925},
    {'key': 'posted-content', 'do

### 1.1.4. Quelles sont les langues des productions ouvertes en santé ? <a class="anchor" id="langue"></a>

In [15]:
json = {
  "size": 0,
  "aggs": {
    "by_is_oa": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
      },
      "aggs": {
        "by_publication_genre": {
          "terms": {
            "field": "lang.keyword"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_is_oa']['buckets']

[{'key': 0,
  'key_as_string': 'false',
  'doc_count': 654731,
  'by_publication_genre': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 1899,
   'buckets': [{'key': 'en', 'doc_count': 487238},
    {'key': 'fr', 'doc_count': 152550},
    {'key': 'es', 'doc_count': 3622},
    {'key': 'de', 'doc_count': 3007},
    {'key': 'it', 'doc_count': 1981},
    {'key': 'ru', 'doc_count': 815},
    {'key': 'ja', 'doc_count': 703},
    {'key': 'pt', 'doc_count': 541},
    {'key': 'ar', 'doc_count': 430},
    {'key': 'nl', 'doc_count': 277}]}},
 {'key': 1,
  'key_as_string': 'true',
  'doc_count': 642317,
  'by_publication_genre': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 1285,
   'buckets': [{'key': 'en', 'doc_count': 556428},
    {'key': 'fr', 'doc_count': 72111},
    {'key': 'es', 'doc_count': 4489},
    {'key': 'pt', 'doc_count': 2527},
    {'key': 'de', 'doc_count': 1602},
    {'key': 'it', 'doc_count': 1335},
    {'key': 'ja', 'doc_count': 983},
    {'key': 'pl',

### 1.1.5. Quel impact le financement a-t-il sur l’ouverture des publications en santé ? <a class="anchor" id="financement"></a>

#### 1.1.5.1. Taux d'ouverture des publications en santé par déclaration de financement par projet

In [20]:
json = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
          }
        }
      },
      "aggs": {
        "by_has_grant": {
          "terms": {
            "field": "has_grant"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
              }
            }
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167082,
  'by_has_grant': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0,
     'key_as_string': 'false',
     'doc_count': 213,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 143},
       {'key': 0, 'key_as_string': 'false', 'doc_count': 70}]}},
    {'key': 1,
     'key_as_string': 'true',
     'doc_count': 27,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 22},
       {'key': 0, 'key_as_string': 'false', 'doc_count': 5}]}}]}},
 {'key': 2019,
  'doc_count': 165572,
  'by_has_grant': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0,
     'key_as_string': 'false',
     'doc_count': 2,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      

#### 1.1.5.2. Répartition par déclaration de financement par projet et par type d’hébergement en santé

In [21]:
json = {
  "size": 0,
  "aggs": {
    "by_is_oa": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type.keyword"
          },
          "aggs": {
            "by_grant_agency": {
              "terms": {
                "field": "grants.agency.keyword"
              }
            }
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_is_oa']['buckets']

[{'key': 0,
  'key_as_string': 'false',
  'doc_count': 654731,
  'by_oa_host_type': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed',
     'doc_count': 654731,
     'by_grant_agency': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 10,
      'buckets': [{'key': 'Agence Nationale de la Recherche', 'doc_count': 1},
       {'key': 'Association France Alzheimer', 'doc_count': 1},
       {'key': 'Cancerfonden', 'doc_count': 1},
       {'key': 'Conseil Regional des Hauts-de-France', 'doc_count': 1},
       {'key': 'Fellowship by the Poitou-Charentes region', 'doc_count': 1},
       {'key': "Fondation pour l'Audition (FR)", 'doc_count': 1},
       {'key': 'Fondation pour la Recherche Medicale', 'doc_count': 1},
       {'key': 'Fondation pour la recherche médicale', 'doc_count': 1},
       {'key': 'IARC-COFUND', 'doc_count': 1},
       {'key': 'Laboratoires B.L.C. Thalgo Cosmetic S.A.',
        'doc_count': 1}]}}]}},
 {'key': 1,
  

### 1.1.6. Quelle transparence dans la déclaration des conflits d'intérêts en santé ? <a class="anchor" id="coi"></a>

In [22]:
json = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "by_has_coi": {
          "terms": {
            "field": "has_coi"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167082,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2019,
  'doc_count': 165572,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2020,
  'doc_count': 163739,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2016,
  'doc_count': 155772,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2017,
  'doc_count': 153697,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2014,
  'doc_count': 146086,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2015,
  'doc_count': 146083,
  'by_has_coi': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': []}},
 {'key': 2013,
  'doc_count': 1459

## 1.2. Les disciplines <a class="anchor" id="publi_disciplines"></a>

### 1.2.1. Quelle est la dynamique d’ouverture de la santé en fonction de ses disciplines ?  <a class="anchor" id="dynamique_ouverture_disciplines"></a>

In [26]:
# pour chaque observation date

json = {
  "size": 0,
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": "bsso_classification.field.keyword"
      },
      "aggs": {
        "by_observation_year": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.observation_date.keyword"
          },
          "aggs": {
            "by_publication_year": {
              "terms": {
                "field": "year"
              },
              "aggs": {
                "by_is_oa": {
                  "terms": {
                    "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_discipline']['buckets']

[{'key': 'Engineering',
  'doc_count': 169827,
  'by_observation_year': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': '2021Q2',
     'doc_count': 169826,
     'by_publication_year': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 148,
      'buckets': [{'key': 2019,
        'doc_count': 21722,
        'by_is_oa': {'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0,
         'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 13071},
          {'key': 0, 'key_as_string': 'false', 'doc_count': 8651}]}},
       {'key': 2018,
        'doc_count': 21393,
        'by_is_oa': {'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0,
         'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 11409},
          {'key': 0, 'key_as_string': 'false', 'doc_count': 9984}]}},
       {'key': 2017,
        'doc_count': 21104,
        'by_is_oa': {'doc_count_error_upper_bound': 0,
         'sum_other

### 1.2.2. Quelles sont les voies d’ouverture choisies par les publications en fonction des disciplines ?  <a class="anchor" id="voie_ouverture_disciplines"></a>

In [27]:
json = {
  "size": 0,
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "publication_year"
          },
          "aggs": {
            "by_oa_host_type": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type.keyword"
              },
              "aggs": {
                "by_discipline": {
                  "terms": {
                    "field": "bsso_classification.field.keyword"
                  }
                }
              }
            }
          }
        }
      }
    }

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 244,
  'by_oa_host_type': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'publisher;repository',
     'doc_count': 92,
     'by_discipline': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 22,
      'buckets': [{'key': 'Clinical Sciences', 'doc_count': 17},
       {'key': 'Multidisciplinary', 'doc_count': 10},
       {'key': 'Oncology and Carcinogenesis', 'doc_count': 10},
       {'key': 'Other Medical and Health Sciences', 'doc_count': 10},
       {'key': 'Agricultural and Veterinary Sciences', 'doc_count': 7},
       {'key': 'Chemical Sciences', 'doc_count': 7},
       {'key': 'Biochemistry and Cell Biology', 'doc_count': 6},
       {'key': 'Cardiorespiratory Medicine and Haematology', 'doc_count': 6},
       {'key': 'Engineering', 'doc_count': 6},
       {'key': 'Microbiology', 'doc_count': 6}]}},
    {'key': 'closed',
     'doc_count': 76,
     'by_discipline': {'doc_count_error_upper_bound': 0,
 

## 1.3. Les éditeurs/plateformes <a class="anchor" id="publi_editeurs"></a>

### 1.3.1. Quelle est la dynamique d’ouverture de la santé chez les éditeurs/plateformes ?  <a class="anchor" id="dynamique_ouverture_editeurs"></a>

In [30]:
filter = "*" # *, Elsevier BV, Springer Science and Business Media LLC, Wiley...
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "publisher" }}, # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
        { "wildcard": { "publisher.keyword": filter }}
      ]
    }
  },
  "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "year"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
              }
            }
          }
        }
      }
    }

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2019,
  'doc_count': 66191,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 66191}]}},
 {'key': 2020,
  'doc_count': 65661,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 65661}]}},
 {'key': 2018,
  'doc_count': 57359,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 57359}]}},
 {'key': 2017,
  'doc_count': 50976,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 50976}]}},
 {'key': 2016,
  'doc_count': 49481,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 49481}]}},
 {'key': 2015,
  'doc_count': 44467,
  'by_is_oa':

### 1.3.2. Quel type d’ouverture est majoritaire chez les éditeurs/plateformes en santé ? <a class="anchor" id="voie_ouverture_editeurs"></a>


In [34]:
filter = "*" # *, Elsevier BV, Springer Science and Business Media LLC, Wiley...
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "publisher" }}, # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
        { "wildcard": { "publisher.keyword": filter }}
      ]
    }
  },
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "year"
          },
          "aggs": {
            "by_oa_colors": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.oa_colors_with_priority_to_publisher.keyword"
              }
            }
          }
        }
      }
    }

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2019,
  'doc_count': 66191,
  'by_oa_colors': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'hybrid', 'doc_count': 36065},
    {'key': 'gold', 'doc_count': 24197},
    {'key': 'diamond', 'doc_count': 5929}]}},
 {'key': 2020,
  'doc_count': 65661,
  'by_oa_colors': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'hybrid', 'doc_count': 31556},
    {'key': 'gold', 'doc_count': 28680},
    {'key': 'diamond', 'doc_count': 5425}]}},
 {'key': 2018,
  'doc_count': 57359,
  'by_oa_colors': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'hybrid', 'doc_count': 30619},
    {'key': 'gold', 'doc_count': 20763},
    {'key': 'diamond', 'doc_count': 5977}]}},
 {'key': 2017,
  'doc_count': 50976,
  'by_oa_colors': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'hybrid', 'doc_count': 27526},
    {'key': 'gold', 'doc_count': 17928},
    {'

### 1.3.3. Quelles sont les politiques d’ouverture des éditeurs/plateformes en santé ? <a class="anchor" id="politique_ouverture_editeurs"></a>

In [36]:
year = 2020 # 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
       { "term": { "year": year }}
      ]
    }
  },
  "aggs": {
    "by_publisher": {
      "terms": {
        "field": "publisher.keyword"
      },
      "aggs": {
        "by_oa_color": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.oa_colors_with_priority_to_publisher.keyword"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publisher']['buckets']

[{'key': 'Elsevier BV',
  'doc_count': 37755,
  'by_oa_color': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 20341},
    {'key': 'hybrid', 'doc_count': 8744},
    {'key': 'green_only', 'doc_count': 6412},
    {'key': 'gold', 'doc_count': 2005},
    {'key': 'diamond', 'doc_count': 253}]}},
 {'key': 'Springer Science and Business Media LLC',
  'doc_count': 16696,
  'by_oa_color': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 6495},
    {'key': 'gold', 'doc_count': 4499},
    {'key': 'hybrid', 'doc_count': 2739},
    {'key': 'green_only', 'doc_count': 2611},
    {'key': 'diamond', 'doc_count': 352}]}},
 {'key': 'Wiley',
  'doc_count': 11360,
  'by_oa_color': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 5447},
    {'key': 'hybrid', 'doc_count': 2862},
    {'key': 'green_only', 'doc_count': 2146},
   

### 1.3.4. Quel est le poids des revues prédatrices dans la dynamique de science ouverte en santé ? <a class="anchor" id="predateur"></a>

In [37]:
json = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "by_predatory": {
          "terms": {
            "field": "predatory_journal"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167082,
  'by_predatory': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 166112},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 970}]}},
 {'key': 2019,
  'doc_count': 165572,
  'by_predatory': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 164833},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 739}]}},
 {'key': 2020,
  'doc_count': 163739,
  'by_predatory': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 163127},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 612}]}},
 {'key': 2016,
  'doc_count': 155772,
  'by_predatory': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 155059},
    {'key': 1, 'key_as_string': 'true'

### 1.3.5. Quelle est la répartition des licences utilisées chez les éditeurs/plateformes en santé ? <a class="anchor" id="licence"></a>


#### 1.3.5.1. Répartition des publications ouvertes en santé par licences utilisées chez les éditeurs/plateformes

In [38]:
filter = "*" # *, Elsevier BV, Springer Science and Business Media LLC, Wiley...
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "publisher" }}, # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
        { "term": { "year": 2020 }},
        { "wildcard": { "publisher.keyword": filter }}
      ]
    }
  },
  "aggs": {
    "by_licence": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.licence_publisher.keyword",
        "missing": "N/A"
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_licence']['buckets']

[{'key': 'cc-by', 'doc_count': 31328},
 {'key': 'no license', 'doc_count': 19656},
 {'key': 'cc-by-nc-nd', 'doc_count': 9106},
 {'key': 'cc-by-nc', 'doc_count': 4143},
 {'key': 'cc-by-nc-sa', 'doc_count': 511},
 {'key': 'publisher-specific', 'doc_count': 474},
 {'key': 'cc-by-sa', 'doc_count': 225},
 {'key': 'implied-oa', 'doc_count': 116},
 {'key': 'cc-by-nd', 'doc_count': 102}]

#### 1.3.5.2. Classement des 10 éditeurs/plateformes les plus importants (en nombre de publications en santé) selon le type de licences utilisées

In [40]:
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "publisher" }}, # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
        { "term": { "year": 2020 }}
      ]
    }
  },
  "aggs": {
    "by_publisher": {
      "terms": {
        "field": "publisher.keyword"
      },
      "aggs": {
        "by_licence": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.licence_publisher.keyword",
            "missing": "N/A"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publisher']['buckets']

[{'key': 'Elsevier BV',
  'doc_count': 11002,
  'by_licence': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'no license', 'doc_count': 6655},
    {'key': 'cc-by-nc-nd', 'doc_count': 3299},
    {'key': 'cc-by', 'doc_count': 1029},
    {'key': 'implied-oa', 'doc_count': 16},
    {'key': 'cc-by-nc', 'doc_count': 1},
    {'key': 'cc-by-nc-sa', 'doc_count': 1},
    {'key': 'cc-by-sa', 'doc_count': 1}]}},
 {'key': 'Springer Science and Business Media LLC',
  'doc_count': 7590,
  'by_licence': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'cc-by', 'doc_count': 6209},
    {'key': 'no license', 'doc_count': 1220},
    {'key': 'cc-by-nc', 'doc_count': 139},
    {'key': 'cc-by-nc-nd', 'doc_count': 10},
    {'key': 'publisher-specific', 'doc_count': 7},
    {'key': 'cc-by-nc-sa', 'doc_count': 4},
    {'key': 'cc-by-sa', 'doc_count': 1}]}},
 {'key': 'MDPI AG',
  'doc_count': 6684,
  'by_licence': {'doc_count_error_upp

### 1.3.6. Quels sont les coûts des publications chez les éditeurs/plateformes en santé ? <a class="anchor" id="apc"></a>


#### 1.3.6.1. Dépenses estimées pour la communauté scientifique, en APC pour la production francaise en santé

In [41]:
json = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "apc": {
          "sum": {
            "field": "amount_apc_EUR"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018, 'doc_count': 167082, 'apc': {'value': 148065359.3903532}},
 {'key': 2019, 'doc_count': 165572, 'apc': {'value': 131956379.0316124}},
 {'key': 2020, 'doc_count': 163739, 'apc': {'value': 91690949.47849084}},
 {'key': 2016, 'doc_count': 155772, 'apc': {'value': 129983930.99725676}},
 {'key': 2017, 'doc_count': 153697, 'apc': {'value': 139513659.64844704}},
 {'key': 2014, 'doc_count': 146086, 'apc': {'value': 98627967.32200432}},
 {'key': 2015, 'doc_count': 146083, 'apc': {'value': 119302493.42411327}},
 {'key': 2013, 'doc_count': 145902, 'apc': {'value': 50660661.03848648}},
 {'key': 2021, 'doc_count': 46603, 'apc': {'value': 20823928.695663452}},
 {'key': 2012, 'doc_count': 3764, 'apc': {'value': 202732.11227416992}}]

#### 1.3.6.2. Distribution des tarifs unitaires des frais de publication par article dans le domaine de la santé

In [42]:
filter = "*" # *, Elsevier BV, Springer Science and Business Media LLC, Wiley...
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "wildcard": { "publisher.keyword": filter }}
      ]
    }
  },
  "aggs": {
    "tarif": {
      "histogram": {
        "field": "amount_apc_EUR",
        "interval": 250
      }
    }
  }
}


requests.post(url, json=json, headers=headers).json()['aggregations']['tarif']['buckets']

[{'key': 0.0, 'doc_count': 4132},
 {'key': 250.0, 'doc_count': 6828},
 {'key': 500.0, 'doc_count': 8685},
 {'key': 750.0, 'doc_count': 11569},
 {'key': 1000.0, 'doc_count': 21834},
 {'key': 1250.0, 'doc_count': 37183},
 {'key': 1500.0, 'doc_count': 47050},
 {'key': 1750.0, 'doc_count': 48223},
 {'key': 2000.0, 'doc_count': 46878},
 {'key': 2250.0, 'doc_count': 41426},
 {'key': 2500.0, 'doc_count': 47022},
 {'key': 2750.0, 'doc_count': 30853},
 {'key': 3000.0, 'doc_count': 25650},
 {'key': 3250.0, 'doc_count': 15395},
 {'key': 3500.0, 'doc_count': 9004},
 {'key': 3750.0, 'doc_count': 5485},
 {'key': 4000.0, 'doc_count': 4782},
 {'key': 4250.0, 'doc_count': 3566},
 {'key': 4500.0, 'doc_count': 3449},
 {'key': 4750.0, 'doc_count': 2214},
 {'key': 5000.0, 'doc_count': 1066},
 {'key': 5250.0, 'doc_count': 839},
 {'key': 5500.0, 'doc_count': 126},
 {'key': 5750.0, 'doc_count': 12},
 {'key': 6000.0, 'doc_count': 31},
 {'key': 6250.0, 'doc_count': 0},
 {'key': 6500.0, 'doc_count': 4},
 {'key':

#### 1.3.6.3. Distribution des tarifs unitaires des frais de publication par article, par année de publication, dans la  santé

violin : 1. one request for density

In [43]:
filter = "*" # *, Elsevier BV, Springer Science and Business Media LLC, Wiley...
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "wildcard": { "publisher.keyword": filter }}
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "tarif": {
          "histogram": {
            "field": "amount_apc_EUR",
            "interval": 250
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167079,
  'tarif': {'buckets': [{'key': 0.0, 'doc_count': 793},
    {'key': 250.0, 'doc_count': 1234},
    {'key': 500.0, 'doc_count': 1369},
    {'key': 750.0, 'doc_count': 1740},
    {'key': 1000.0, 'doc_count': 3078},
    {'key': 1250.0, 'doc_count': 4514},
    {'key': 1500.0, 'doc_count': 6543},
    {'key': 1750.0, 'doc_count': 6455},
    {'key': 2000.0, 'doc_count': 5628},
    {'key': 2250.0, 'doc_count': 6758},
    {'key': 2500.0, 'doc_count': 6790},
    {'key': 2750.0, 'doc_count': 4749},
    {'key': 3000.0, 'doc_count': 5127},
    {'key': 3250.0, 'doc_count': 2976},
    {'key': 3500.0, 'doc_count': 1800},
    {'key': 3750.0, 'doc_count': 1218},
    {'key': 4000.0, 'doc_count': 1040},
    {'key': 4250.0, 'doc_count': 605},
    {'key': 4500.0, 'doc_count': 1106},
    {'key': 4750.0, 'doc_count': 146},
    {'key': 5000.0, 'doc_count': 328},
    {'key': 5250.0, 'doc_count': 230},
    {'key': 5500.0, 'doc_count': 18},
    {'key': 5750.0, 'doc_count': 3}

violin : second request for percentiles

In [44]:
filter = "*" # *, Elsevier BV, Springer Science and Business Media LLC, Wiley...
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "wildcard": { "publisher.keyword": filter }}
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "tarif_percentiles": {
          "percentiles": {
            "field": "amount_apc_EUR"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167079,
  'tarif_percentiles': {'values': {'1.0': 200.0,
    '5.0': 698.6868107940392,
    '25.0': 1628.25700871588,
    '50.0': 2304.954024672894,
    '75.0': 2907.739643824001,
    '95.0': 4082.890625,
    '99.0': 4949.28515625}}},
 {'key': 2019,
  'doc_count': 165563,
  'tarif_percentiles': {'values': {'1.0': 303.6820091247559,
    '5.0': 770.3993416444773,
    '25.0': 1588.010837009975,
    '50.0': 2117.5612650360913,
    '75.0': 2689.4576089910074,
    '95.0': 3810.9482509116265,
    '99.0': 4803.0751953125}}},
 {'key': 2020,
  'doc_count': 163733,
  'tarif_percentiles': {'values': {'1.0': 298.25530074368993,
    '5.0': 794.5064550608531,
    '25.0': 1580.1864667948137,
    '50.0': 1898.8248131547425,
    '75.0': 2632.9839191460846,
    '95.0': 3743.219970703125,
    '99.0': 4445.0}}},
 {'key': 2016,
  'doc_count': 155771,
  'tarif_percentiles': {'values': {'1.0': 221.66000366210938,
    '5.0': 785.6377353454425,
    '25.0': 1696.5882190925233,
    '5

## 1.4. Les archives <a class="anchor" id="publications_archives"></a>

### 1.4.1. Quelle est la dynamique d’ouverture de la santé parmi les archives ? <a class="anchor" id="publications_archives_ouverture"></a>

In [45]:
# pour chaque observation year

# pour le calcul, il faut faire le ration (repository + "publisher;repository") / (doc_count)

json = {
  "size": 0,

 
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "year"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type.keyword"
              }
            }
          }
        }
      }
    }


requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167082,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 80916},
    {'key': 'publisher;repository', 'doc_count': 33752},
    {'key': 'repository', 'doc_count': 28804},
    {'key': 'publisher', 'doc_count': 23610}]}},
 {'key': 2019,
  'doc_count': 165572,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 71118},
    {'key': 'publisher;repository', 'doc_count': 37679},
    {'key': 'publisher', 'doc_count': 28514},
    {'key': 'repository', 'doc_count': 28261}]}},
 {'key': 2020,
  'doc_count': 163739,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'closed', 'doc_count': 69555},
    {'key': 'publisher;repository', 'doc_count': 40677},
    {'key': 'repository', 'doc_count': 28517},
    {'key': 'publisher', 'doc_count': 24990}]}},
 {'key': 2016,
  'doc_count': 1

### 1.4.2. Quelles archives ouvertes sont les plus utilisées en santé ? <a class="anchor" id="publications_archives_utilisation"></a>

In [51]:
AGG_SIZE=50
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "repository"
        }
      }]
    }
  },

      "aggs": {
        "by_repository": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.repositories.keyword",
            "missing": "N/A",
            "size": AGG_SIZE
          }
        }
      }
    }

requests.post(url, json=json, headers=headers).json()['aggregations']['by_repository']['buckets']

[{'key': 'HAL', 'doc_count': 201808},
 {'key': 'europepmc.org', 'doc_count': 122649},
 {'key': 'arXiv.org', 'doc_count': 70557},
 {'key': 'N/A', 'doc_count': 35842},
 {'key': 'CiteSeerX.psu', 'doc_count': 14017},
 {'key': 'edpsciences.org', 'doc_count': 10850},
 {'key': 'pure.atira.dk', 'doc_count': 8790},
 {'key': 'oai', 'doc_count': 6891},
 {'key': 'RePEc', 'doc_count': 6038},
 {'key': 'archimer.ifremer.fr', 'doc_count': 5690},
 {'key': 'osti.gov', 'doc_count': 5542},
 {'key': 'digital.csic.es', 'doc_count': 5203},
 {'key': 'zenodo.org', 'doc_count': 5145},
 {'key': 'eprints.ucl.ac.uk.OAI2', 'doc_count': 4913},
 {'key': 'oatao.univ-toulouse.fr', 'doc_count': 4832},
 {'key': 'revues.org', 'doc_count': 4391},
 {'key': 'spiral.imperial.ac.uk', 'doc_count': 4334},
 {'key': 'eprints.whiterose.ac.uk', 'doc_count': 4107},
 {'key': 'pure.ed.ac.uk', 'doc_count': 4080},
 {'key': 'cds.cern.ch', 'doc_count': 3683},
 {'key': 'www.zora.uzh.ch', 'doc_count': 3326},
 {'key': 'DiVA.org', 'doc_count':

### 1.4.3. Quelle est la dynamique de dépôt par archive ouverte en santé ? <a class="anchor" id="publications_archives_dynamique"></a>

In [52]:
json = {
  "size": 0,
      "query": {
    "bool": {
      "filter": [{
        "term": {
          f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "repository"
        }
      }]
    }
  },
  "aggs": {
    "by_repository": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.repositories.keyword",
        "missing": "N/A",
        "size": 12
      },
    
          "aggs": {
            "by_publication_year": {
              "terms": {
                "field": "year"
              }
            }
          }
        }
      }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_repository']['buckets']

[{'key': 'HAL',
  'doc_count': 201808,
  'by_publication_year': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 391,
   'buckets': [{'key': 2019, 'doc_count': 31646},
    {'key': 2020, 'doc_count': 30891},
    {'key': 2018, 'doc_count': 27930},
    {'key': 2017, 'doc_count': 25234},
    {'key': 2016, 'doc_count': 22894},
    {'key': 2015, 'doc_count': 20419},
    {'key': 2014, 'doc_count': 18375},
    {'key': 2013, 'doc_count': 17044},
    {'key': 2021, 'doc_count': 5814},
    {'key': 2012, 'doc_count': 875}]}},
 {'key': 'europepmc.org',
  'doc_count': 122649,
  'by_publication_year': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 85,
   'buckets': [{'key': 2019, 'doc_count': 18910},
    {'key': 2018, 'doc_count': 18508},
    {'key': 2017, 'doc_count': 17382},
    {'key': 2016, 'doc_count': 16430},
    {'key': 2015, 'doc_count': 14545},
    {'key': 2014, 'doc_count': 13157},
    {'key': 2013, 'doc_count': 12269},
    {'key': 2020, 'doc_count': 11006},
    {'k

### 1.4.4. Quelle place occupe HAL dans la dynamique des archives ouvertes en santé ? <a class="anchor" id="publications_archives_hal"></a>

requête à revoir !! il faut avoir un filtre avec ou sans HAL

In [53]:
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "repository" }}
      ]
    }
  },
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.repositories.keyword",
        "missing": "N/A"
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_discipline']

{'doc_count_error_upper_bound': 0,
 'sum_other_doc_count': 273514,
 'buckets': [{'key': 'HAL', 'doc_count': 201808},
  {'key': 'europepmc.org', 'doc_count': 122649},
  {'key': 'arXiv.org', 'doc_count': 70557},
  {'key': 'N/A', 'doc_count': 35842},
  {'key': 'CiteSeerX.psu', 'doc_count': 14017},
  {'key': 'edpsciences.org', 'doc_count': 10850},
  {'key': 'pure.atira.dk', 'doc_count': 8790},
  {'key': 'oai', 'doc_count': 6891},
  {'key': 'RePEc', 'doc_count': 6038},
  {'key': 'archimer.ifremer.fr', 'doc_count': 5690}]}

In [63]:
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { f"oa_details.{LATEST_OBSERVATION_DATE}.oa_host_type": "repository" }}
      ]
    }
  },
  "aggs": {
    "by_discipline": {
      "terms": {
        "field": f"oa_details.{LATEST_OBSERVATION_DATE}.repositories.keyword",
        "missing": "N/A"
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()#['aggregations']['by_discipline']

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'by_discipline': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 273514,
   'buckets': [{'key': 'HAL', 'doc_count': 201808},
    {'key': 'europepmc.org', 'doc_count': 122649},
    {'key': 'arXiv.org', 'doc_count': 70557},
    {'key': 'N/A', 'doc_count': 35842},
    {'key': 'CiteSeerX.psu', 'doc_count': 14017},
    {'key': 'edpsciences.org', 'doc_count': 10850},
    {'key': 'pure.atira.dk', 'doc_count': 8790},
    {'key': 'oai', 'doc_count': 6891},
    {'key': 'RePEc', 'doc_count': 6038},
    {'key': 'archimer.ifremer.fr', 'doc_count': 5690}]}}}

_Idée : Should post process to group / sum all repositories other than 'HAL'_

## 1.5. Les affiliations <a class="anchor" id="publications_affiliations"></a>

### 1.5.1. Taux d’ouverture des publications françaises, dans le domaine de la santé, par millésime tous types d’établissements confondus <a class="anchor" id="publications_affiliations_ouverture"></a>

In [54]:
filter = "*" # *, university, hospital, cnrs, inserm
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "wildcard": { "french_affiliations_types": filter }}
      ]
    }
  },
      "aggs": {
        "by_publication_year": {
          "terms": {
            "field": "year"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
              }
            }
          }
        }
      }
    }

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 224,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 156},
    {'key': 0, 'key_as_string': 'false', 'doc_count': 68}]}},
 {'key': 2021,
  'doc_count': 100,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 57},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 43}]}},
 {'key': 2020,
  'doc_count': 11,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 7},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 4}]}},
 {'key': 2019,
  'doc_count': 4,
  'by_is_oa': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 3},
    {'key': 0, 'key_as_string': 'false', 'doc_count': 1}]}}]

### 1.5.2. Evolution du taux d’ouverture des publications en santé par types d’établissement entre millésimes <a class="anchor" id="publications_affiliations_evolution"></a>

_Warning: Les données ne sont pas encore prêtes._

### 1.5.3. Quel impact le pays d’affiliation des auteurs a-t-il sur le taux d’ouverture en santé ? <a class="anchor" id="publications_affiliations_impact"></a>

In [55]:
json = {
  "size": 0,
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "by_author_useful_rank_fr": {
          "terms": {
            "field": "author_useful_rank_fr"
          },
          "aggs": {
            "by_is_oa": {
              "terms": {
                "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
              }
            }
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_publication_year']['buckets']

[{'key': 2018,
  'doc_count': 167082,
  'by_author_useful_rank_fr': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0,
     'key_as_string': 'false',
     'doc_count': 166904,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 86052},
       {'key': 0, 'key_as_string': 'false', 'doc_count': 80852}]}},
    {'key': 1,
     'key_as_string': 'true',
     'doc_count': 178,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 114},
       {'key': 0, 'key_as_string': 'false', 'doc_count': 64}]}}]}},
 {'key': 2019,
  'doc_count': 165572,
  'by_author_useful_rank_fr': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0,
     'key_as_string': 'false',
     'doc_count': 165570,
     'by_is_oa': {'doc_count_error_upper_bound': 0

### 1.5.4. Classement des 10 pays d'affiliation des auteurs de rang utile avec lesquels la France collabore le plus, selon le taux d’accès ouvert de leurs publications en santé <a class="anchor" id="publications_affiliations_classement"></a>

In [60]:
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [
        { "term": { "author_useful_rank_fr": True }}
      ]
    }
  },
  "aggs": {
    "by_country": {
      "terms": {
        "field": "detected_countries.keyword",
        "exclude": "fr"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": f"oa_details.{LATEST_OBSERVATION_DATE}.is_oa"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()#['aggregations']['by_country']['buckets']

{'took': 29,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 263, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'by_country': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 58,
   'buckets': [{'key': 'us',
     'doc_count': 19,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 18},
       {'key': 0, 'key_as_string': 'false', 'doc_count': 1}]}},
    {'key': 'gb',
     'doc_count': 15,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 10},
       {'key': 0, 'key_as_string': 'false', 'doc_count': 5}]}},
    {'key': 'es',
     'doc_count': 12,
     'by_is_oa': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 1, 'key_as_string': 'true', '